In [1]:
import define_transformations_circular_economy as dtc
import define_transformations_energy as dte

import os, os.path
import numpy as np
import pandas as pd
#from model_attributes import *
import model_attributes as ma
from attribute_table import AttributeTable
import model_afolu as mafl
import model_ippu as mi
import model_circular_economy as mc
import model_electricity as ml
import model_energy as me
import model_socioeconomic as se
from model_socioeconomic import Socioeconomic
import setup_analysis as sa
import sisepuede_models as sm
import support_classes as sc
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
from typing import Union
import inspect
import ingestion as ing
import logging
from sisepuede_file_structure import *
import transformations_base_circular_economy as tbc
import transformations_base_energy as tbe

importlib.reload(ma)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(mafl)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(me)
importlib.reload(se)
importlib.reload(tbc)
importlib.reload(tbe)


import sisepuede as ssp
def _setup_logger(namespace: str, fn_out: Union[str, None] = None) -> None:
    global logger
    
    format_str = "%(asctime)s - %(levelname)s - %(message)s"
    # configure
    if fn_out is not None:
        logging.basicConfig(
            filename = fn_out,
            filemode = "w",
            format = format_str,
            level = logging.DEBUG
        )
    else:
        logging.basicConfig(
            format = format_str,
            level = logging.DEBUG
        )
        
    logger = logging.getLogger(namespace)
    # create console handler and set level to debug
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    # create formatter
    formatter = logging.Formatter(format_str)
    # add formatter to ch
    ch.setFormatter(formatter)
    # add ch to logger
    logger.addHandler(ch)

    return logger

_setup_logger(__name__, os.path.join(os.getcwd(), "log_sisepuede_transformations_energy.log"))

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:2322: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:2322: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


<Logger __main__ (DEBUG)>

In [2]:

##########################
#   LOAD INPUT TABLES    #
##########################

warnings.filterwarnings("ignore")
importlib.reload(ml)
importlib.reload(me)


regions = sc.Regions(sa.model_attributes)
time_periods = sc.TimePeriods(sa.model_attributes)

##  setup location of calibrated files by sector

df_fake_data = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))
dir_calibs = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo"

# setup output templates 
dir_template_inputs = os.path.join(dir_calibs, "template_inputs")
os.makedirs(dir_template_inputs, exist_ok = True) if not os.path.exists(dir_template_inputs) else None
dict_template_file_paths = {}

dict_calibration_file_paths = {
    "af": os.path.join(dir_calibs, "af", "data_complete_future_2022_09_30_test_updated.csv"),
    "ce": os.path.join(dir_calibs, "ce", "data_complete_future_2022_12_13_test.csv"),
    "en": os.path.join(dir_calibs, "en", "data_complete_future_2023_02_24.csv"),#"data_complete_future_2022_12_09_test.csv"),
    "ip": os.path.join(dir_calibs, "ip", "data_ippu_2023_01_16_fixed_cleaned.csv"),
    # use most recent for socioeconomic
    "se": os.path.join(dir_calibs, "en", "data_complete_future_2023_02_24.csv")
}

# TEMP FOR INDIA
running_out_of_lac = False
if running_out_of_lac:
    print("READING NON-STANDARD DATA")
    for key in dict_calibration_file_paths.keys():
        dict_calibration_file_paths.update({key: os.path.join(dir_calibs, "all", "Data_Input_2023_05_12_MNE.csv")})

"""
dict_calibration_file_paths = {
    "af": os.path.join(dir_calibs, "all", "datos_calibrados_20230221.csv"),
    "ce": os.path.join(dir_calibs, "all", "datos_calibrados_20230221.csv"),
    "en": os.path.join(dir_calibs, "all", "datos_calibrados_20230221.csv"),#"data_complete_future_2022_12_09_test.csv"),
    "ip": os.path.join(dir_calibs, "all", "datos_calibrados_20230221.csv"),
    # use most recent for socioeconomic
    "se": os.path.join(dir_calibs, "all", "datos_calibrados_20230221.csv")
}
""";


# initialize 
all_regions = None
dict_isos = {}
dict_calibration_tables = {}
dict_replace_iso = sa.model_attributes.dict_attributes.get("region").field_maps.get("iso_alpha_3_to_region")
dict_sets = {}
field_country = "country"
field_iso = "iso_code3"
field_region = "nation"
field_year = "year"
fields_drop = [field_iso, field_region, field_year] # only apply later
field_time_period = sa.model_attributes.dim_time_period

# get some attributes
attr_region = sa.model_attributes.dict_attributes.get("region")
attr_sector = sa.model_attributes.dict_attributes.get("abbreviation_sector")
attr_strat = sa.model_attributes.dict_attributes.get(f"dim_{sa.model_attributes.dim_strategy_id}")
attr_time_period = sa.model_attributes.dict_attributes.get(f"dim_{sa.model_attributes.dim_time_period}")

# dictionary and attribute derivaties
dict_strat_to_strat_id = attr_strat.field_maps.get("strategy_to_strategy_id")
dict_repl_time_period = attr_time_period.field_maps.get(f"year_to_{sa.model_attributes.dim_time_period}")
strat_base = int(attr_strat.table[attr_strat.table["baseline_strategy_id"] == 1][attr_strat.key])
# map each country to ISO code 3 and each code to 
dict_country_to_iso = dict((k, v.upper()) for k, v in attr_region.field_maps.get(f"{attr_region.key}_to_iso_alpha_3").items())
dict_iso_to_country = sf.reverse_dict(dict_country_to_iso)
all_iso = list(dict_iso_to_country.keys())

# some time period oriented tools
dict_year_to_time_period = attr_time_period.field_maps.get(f"year_to_{attr_time_period.key}")
def year_to_tp(
    year: int
) -> int:

    m = min((dict_year_to_time_period.keys()))
    m_v = dict_year_to_time_period.get(m)

    diff = m - m_v

    return dict_year_to_time_period.get(year, year - diff)

# initialize relevant models
model_afolu = mafl.AFOLU(sa.model_attributes)
model_energy = me.NonElectricEnergy(sa.model_attributes)
model_elec = ml.ElectricEnergy(
    sa.model_attributes, 
    sa.dir_jl,
    sa.dir_ref_nemo,
    initialize_julia = False
)



#
#    SOME FUNCTIONS    #
#

# function to verify time periods
def check_time_periods(
    df_in: pd.DataFrame,
    attribute_time_period: AttributeTable,
    fields_group_by: List[str]
) -> pd.DataFrame:
    """
    Check a data frame to ensure that a record exists for each time period
        for the grouping specified in fields_group_by
    """
    
    # check fields
    fields_group_by = [x for x in fields_group_by if x in df_in.columns]
    if len(fields_group_by) == 0:
        return None
    
    field_time_period = attribute_time_period.key
    
    dfs = df_in.groupby(fields_group_by)
    df_out = []
    for df in dfs:
        i, df = df
        
        tps = sorted(list(df[field_time_period]))
        tps_set = sorted(list(set(tps)))
        if (tps == tps_set) & (set(tps) == set(attribute_time_period.key_values)):
            df_out.append(df) 
    
    return pd.concat(df_out, axis = 0).reset_index(drop = True)



def scale_vars(
    df_in: pd.DataFrame,
    dict_scale: Dict[str, float],
    model_attributes: ModelAttributes
) -> pd.DataFrame:
    """
    Scale model variables k in a data frame according to 
        dict_scale = {k: scalar}
    """
    
    df_out = df_in.copy()
    for modvar in dict_scale.keys():
        subsec = model_attributes.get_variable_subsector(modvar)
        fields = model_attributes.build_varlist(
            subsec,
            modvar
        ) if (subsec is not None) else None
        
        if fields is not None:
            df_out[fields] *= dict_scale.get(modvar)
        
    return df_out



# set up the MSP Max Prod growth rate for hydropower--basically, stop it 
def transformation_change_msp_max(
    df_input: pd.DataFrame,
    dict_cat_to_vector: Dict[str, float],
    model_attributes: ma.ModelAttributes,
    model_electricity: ml.ElectricEnergy,
    drop_flag: Union[int, float, None] = None,
    model_energy: Union[me.NonElectricEnergy, None] = None,
    **kwargs
) -> pd.DataFrame:
    """ 
    Modify the maximum growth in production by electricity generation technology
        in serve of MinShareProduction specifications. Used to prevent a
        technology's estimated production from growing (or to place a cap on 
        growth in production) after the last zero-valued time-period
        associated with `vec_ramp`. 
    
    For exampmle, can be used to prevent hydropoower prodction fom increasing 
        after a given time period (based on `vec_ramp`).

    Function Arguments
    ------------------
    - df_input: input data frame containing baseline trajectories
    - dict_cat_to_vector: dictionary mapping a technology category to two an 
        input vector. The vector uses a drop flag to (generally -999) to identify
        time periods that are not subject to an MSP Max Prod; other values greater
        than 0 are used to identify the maximum deviation from the *last time 
        period with a non-drop flag*, entered as a proportion.
    - model_attributes: ModelAttributes object used to call strategies/variables
    - model_electricity: Electricity and Fuel Production model used to call 
        variables
    - vec_ramp: ramp vec used for implementation

    Keyword Arguments
    -----------------
    - drop_flag: value in 
        model_electricity.modvar_entc_max_elec_prod_increase_for_msp used to 
        signal the presence of no constraint. Defaults to 
        model_electricity.drop_flag_tech_capacities if None
    - field_region: field in df_input that specifies the region
    - regions_apply: optional set of regions to use to define strategy. If None,
        applies to all regions.
    - strategy_id: optional specification of strategy id to add to output
        dataframe (only added if integer)
    """
    
    drop_flag = model_electricity.drop_flag_tech_capacities if not sf.isnumber(drop_flag) else drop_flag
    
    # check for variables and initialize fields_check as drops
    df_out = df_input.copy()
    fields_check = model_attributes.build_varlist(None, model_electricity.modvar_entc_max_elec_prod_increase_for_msp)
    df_out[fields_check] = drop_flag
    
    for cat, vec in dict_cat_to_vector.items():
        
        dict_trans = {
            model_elec.modvar_entc_max_elec_prod_increase_for_msp: {
                "bounds": (drop_flag, np.inf),
                "categories": [cat],
                "magnitude": vec,
                "magnitude_type": "vector_specification",
                "vec_ramp": vec
            }
        }
        
        # call general transformation
        df_out = tbe.transformation_general(
            df_out,
            model_attributes,
            dict_trans,
            model_energy = model_electricity.model_energy,
            **kwargs
        )

        
    return df_out







########################
###                  ###
###    BEGIN TEMP    ###
###                  ###
########################

###
###    TEMPORARY: OVERWRITE CAPACITIES WITH EARLIER DATA
###

df_residual_capacity_overwrite = pd.read_csv(sa.fp_csv_nemomod_residual_capacity_inputs)
df_residual_capacity_overwrite = df_residual_capacity_overwrite[
    df_residual_capacity_overwrite["year"].isin(dict_repl_time_period.keys())
]

dict_replace_iso_rev = sf.reverse_dict(dict_replace_iso)
dict_replace_iso_rev = dict((k, v.upper()) for k, v in dict_replace_iso_rev.items())
df_residual_capacity_overwrite["year"].replace(dict_repl_time_period, inplace = True)

df_residual_capacity_overwrite.drop([attr_region.key], axis = 1, inplace = True)
df_residual_capacity_overwrite.rename(columns = {
    "year": sa.model_attributes.dim_time_period,
    "iso_alpha_3": field_iso
}, inplace = True)


###
###  TEMPORARY: OVERWRITE AFOLU CLIMATE DATA AND SOC
###

# KCC
df_kcc = pd.read_csv(sa.fp_csv_climate_fields_by_country_simple)
df_kcc.rename(
    columns = {regions.field_iso: field_iso}, 
    inplace = True
)
#df_kcc[field_iso] = df_kcc[field_country].replace(dict_country_to_iso)
#df_kcc.drop([field_country], axis = 1, inplace = True)

# SOC
df_soc = pd.read_csv(sa.fp_csv_soc_fields_by_country_simple)
df_soc.rename(
    columns = {regions.field_iso: field_iso}, 
    inplace = True
)

#df_soc[field_iso] = df_soc[field_country].replace(dict_country_to_iso)
#df_soc.drop([field_country], axis = 1, inplace = True)

df_climate = pd.merge(df_kcc, df_soc, how = "inner")


###
###  TEMPORARY: OVERWRITE SOME MORE ENERGY DATA
###

fp_data_overwrite = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo/en/data_complete_future_2023_05_12.csv"
df_overwrite = pd.read_csv(fp_data_overwrite)
# fill in Infs
df_overwrite.replace([np.inf, -np.inf], 0.0, inplace = True)

fields_drop = [x for x in df_overwrite.columns if ("gas_furnace" in x) or ("gas_petroleum_liquid" in x)]
fields_drop += [x for x in df_overwrite.columns if ("residual_capacity" in x)]
fields_drop += [x for x in ["nemomod_entc_total_annual_max_capacity_pp_hydropower_gw"] if x in df_overwrite.columns]
fields_drop_ow = [x for x in df_overwrite.columns if ("variable_cost" in x) and ("per_gw" in x)]

df_overwrite = time_periods.years_to_tps(
    df_overwrite,
    field_year = "Year"
)
df_overwrite[sa.model_attributes.dim_time_period] = df_overwrite["Year"].apply(year_to_tp)
df_overwrite.drop(fields_drop, axis = 1, inplace = True) if (len(fields_drop) > 0) else None

# merge/interpolate to deal with any missing years
df_all_rows = sf.explode_merge(
    attr_time_period.table[[attr_time_period.key]].drop_duplicates(),
    df_overwrite[[field_iso]].drop_duplicates()
)
df_overwrite = pd.merge(
    df_all_rows, 
    df_overwrite, 
    how = "left"
).interpolate().drop(["Year", "Nation"], axis = 1).sort_values(by = [field_iso, attr_time_period.key]).reset_index(drop = True)
"""
# TEMPORARY: FIX MINOR DEVIATIONS IN SUM
if fp_data_overwrite == "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo/en/data_complete_future_2023_04_08.csv":
    
    flds_overwrite = [
        "frac_trns_fuelmix_road_heavy_regional_hydrocarbon_gas_liquids",
        "frac_trns_fuelmix_road_light_hydrocarbon_gas_liquids"
    ]
    
    for fld in flds_overwrite:
        df_overwrite[fld] = 0.0
        
    print(f"overwriting from file {fp_data_overwrite}: scaling `frac_trns_fuelmix_road_heavy_regional_$CAT-FUEL$` to rebalance minor deviations from 1...")
    vl = [
        "frac_trns_fuelmix_road_heavy_regional_biofuels",
        "frac_trns_fuelmix_road_heavy_regional_diesel",
        "frac_trns_fuelmix_road_heavy_regional_electricity",
        "frac_trns_fuelmix_road_heavy_regional_gasoline",
        "frac_trns_fuelmix_road_heavy_regional_hydrocarbon_gas_liquids",
        "frac_trns_fuelmix_road_heavy_regional_hydrogen",
        "frac_trns_fuelmix_road_heavy_regional_natural_gas"
    ]
    
    vec_tot = np.array(df_overwrite[vl]).sum(axis = 1)
    
    for field in vl:
        df_overwrite[field] = np.array(df_overwrite[field])/vec_tot
        
        
    print(f"overwriting from file {fp_data_overwrite}: scaling `frac_trns_fuelmix_road_light_$CAT-FUEL$` to rebalance minor deviations from 1...")
    vl = [
        "frac_trns_fuelmix_road_light_biofuels",
        "frac_trns_fuelmix_road_light_diesel",
        "frac_trns_fuelmix_road_light_electricity",
        "frac_trns_fuelmix_road_light_gasoline",
        "frac_trns_fuelmix_road_light_hydrocarbon_gas_liquids",
        "frac_trns_fuelmix_road_light_hydrogen"
    ]
    
    vec_tot = np.array(df_overwrite[vl]).sum(axis = 1)
    
    for field in vl:
        df_overwrite[field] = np.array(df_overwrite[field])/vec_tot
""";
        


###
###    TEMPORARY: OVERWRITE INITIAL PRODUCTION
###

df_prodvalues = pd.read_csv("/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/ippu/ProductionValues_2023_02_17.csv")
field_tp = sa.model_attributes.dim_time_period
df_tp = sa.model_attributes.dict_attributes.get(f"dim_{field_tp}").table[[field_tp]]
df_prodvalues = sf.explode_merge(df_prodvalues, df_tp) if (field_tp not in df_prodvalues.columns) else df_prodvalues
df_prodvalues["nation"].replace(dict_replace_iso_rev, inplace = True)
df_prodvalues.rename(
    columns = {
        "nation": field_iso
    }, 
    inplace = True
)


###
###    TEMPORARY: OVERWRITE SELECT EFFICIENCY FACTORS REPRESENTING IMPROVEMENTS IN ELECTRIFYING HEAT ENERGY TO 
###

fields_overwrite = sa.model_attributes.build_varlist(
    sa.model_attributes.subsec_name_enfu,
    "Average Industrial Energy Fuel Efficiency Factor",
    restrict_to_category_values = ["fuel_electricity"]
) + sa.model_attributes.build_varlist(
    sa.model_attributes.subsec_name_scoe,
    "SCOE Efficiency Factor for Heat Energy from Electricity"
)
df_effic_factors = df_fake_data[[field_time_period] + fields_overwrite]



###
###    TEMPORARY: CAP MAX TECH CAPACITY FOR HYDROPOWER
###

df_hydro_cap_synthetic = pd.read_csv(sa.fp_csv_nemomod_hydropower_max_tech_capacity)



###
###    TEMPORARY: READ IN MinShareProduction BASELINE VALUES
###


df_min_share_production = pd.read_csv(
    sa.fp_csv_nemomod_minimum_share_of_production_baselines
).drop(["year"], axis = 1)

if False:
    # temporary modification - try cutting the MSP in hal for the first 6 years (while capping new investments)
    df_min_share_production.loc[
        [x < 6 for x in list(df_min_share_production[sa.model_attributes.dim_time_period])], 
        [x for x in df_min_share_production.columns if "nemomod" in x]
    ] = df_min_share_production.loc[
        [x < 6 for x in list(df_min_share_production[sa.model_attributes.dim_time_period])], 
        [x for x in df_min_share_production.columns if "nemomod" in x]
    ] * 0.0
    
def rescale_msps(
    df_msp: pd.DataFrame,
    field_time_period: str = sa.model_attributes.dim_time_period,
    min_total: Union[float, None] = 1.0,
    time_periods_apply: Union[List[int], None] = None
) -> pd.DataFrame:
    """
    Rescale minimum shares of production to meet a minimum total
    
    Function Arguments
    ------------------
    - df_msp: data frame containing variables associatedion with
        model_elec.modvar_entc_nemomod_min_share_production that are 
        to be rescaled (potentially) to meet a minimum total
    
    Keyword Arguments
    -----------------
    - field_time_period: field containing the time period
    - min_total: minimum total for minimum shares of production
    - time_periods_apply: time periods to apply minimum to 
    """
    
    if min_total is None:
        return df_msp
    
    # get time periods that are to be rescaled
    time_periods_apply = (
        list(df_msp[field_time_period].unique()) 
        if (time_periods_apply is None)
        else time_periods_apply
    )
    
    
    # variables to rescale
    vars_rescale = sa.model_attributes.build_varlist(
        None,
        model_elec.modvar_entc_nemomod_min_share_production
    )
    vars_rescale = [x for x in df_msp.columns if x in vars_rescale]

    # split off appropriate years
    df_rescale = df_msp[df_msp[field_time_period].isin(time_periods_apply)].copy().reset_index(drop = True)
    df_ignore = df_msp[~df_msp[field_time_period].isin(time_periods_apply)].copy().reset_index(drop = True)
    
    # get totals and finally rescale
    vec_total = np.array(df_rescale[vars_rescale].sum(axis = 1))
    vec_min = min_total*np.ones(len(df_rescale))
    vec_scale = sf.vec_bounds(vec_min/vec_total, (1, np.inf))
    df_rescale[vars_rescale] = sf.do_array_mult(np.array(df_rescale[vars_rescale]), vec_scale)
    
    
    df_msp = (
        pd.concat([df_rescale, df_ignore], axis = 0)
        .sort_values(by = [field_iso, field_time_period])
        .reset_index(drop = True)
    )
    
    
    return df_msp, vars_rescale

# RESCALE MSP AT EARLY TIME PERIODS TO BE MORE SIGNFICANT (TEMPORARY SOLUTION 04-21-23--REAL SOLUTION SHOULD INCLUDE BETTER RESIDUAL CAPACITY/CAPACITY FACTORS)
df_min_share_production, vars_rescale = rescale_msps(df_min_share_production, time_periods_apply = list(range(6)))
    


###
###    TEMPORARY: READ IN TRANSMISSION LOSSES BY COUNTRY
###

df_transmission_losses = pd.read_csv(
    sa.fp_csv_nemomod_transmission_losses
).drop(["year"], axis = 1)
df_transmission_losses = df_transmission_losses[
    df_transmission_losses[attr_time_period.key].isin(attr_time_period.key_values)
].reset_index(drop = True)



###
###    TEMPORARY: READ IN FUEL COSTS
###

df_fuel_costs = pd.read_csv(sa.fp_csv_nemomod_fuel_costs)



###
###    TEMPORARY: GET TECHNOLOGY MAX CAPACITY INVESTMENTS
###

fields_overwrite = sa.model_attributes.build_varlist(
    sa.model_attributes.subsec_name_entc,
    model_elec.modvar_entc_nemomod_total_annual_max_capacity_investment
)
df_max_cap = df_fake_data[[field_time_period] + fields_overwrite]



###
###  TEMPORARY OVERWRITE OF WOOD FUEL FRACTIONS FROM RECYCLED WOOD 
###


modvars_frac = model_energy.modvars_inen_list_fuel_fraction
dict_cp_wood = {}

# set cateogories
attr_ind = sa.model_attributes.dict_attributes.get("cat_industry")
dict_ind_to_recycling_target = attr_ind.field_maps.get("cat_industry_to_target_cat_industry_to_adjust_with_recycling")

# get target category (wood) and source from recycling
cat_target = model_afolu.cat_ippu_wood
cat_source = None
for k in dict_ind_to_recycling_target.keys():
    cat_source = k if (dict_ind_to_recycling_target.get(k) == f"``{cat_target}``") else cat_source


if cat_source is not None:
    for modvar in modvars_frac:

        cats = sa.model_attributes.get_variable_categories(modvar)
        subsec = sa.model_attributes.get_variable_subsector(modvar)
        
        if (cat_source in cats) and (cat_target in cats):

            field_source = sa.model_attributes.build_varlist(
                subsec,
                modvar,
                restrict_to_category_values = [cat_source]
            )[0]

            field_target = sa.model_attributes.build_varlist(
                subsec,
                modvar,
                restrict_to_category_values = [cat_target]
            )[0]

            dict_cp_wood.update({field_source: field_target})

            
###
###   TEMP: DEFINE FUNCTION TO ADD BIOFUELS (BIOMASS) TO MANUFACTURING INDUSTRIES (excl electronics, cement, metals, glass, lime_and_carbonites)
###
###   04-21-2023
###

cats_inen_with_biofuels = [
    "chemicals",
    "other_industries",
    "other_product_manufacturing",
    "paper",
    "plastic",
    "textiles",
    "wood"
]


def modify_input_inen_fracs(
    df_energy_trajectories_in: pd.DataFrame,
    dict_fuel_info: Dict,
    cats_to_apply: Union[List[str], None] = None,
    field_time_period: str = sa.model_attributes.dim_time_period,
    regions: Union[List, None] = None
) -> pd.DataFrame:
    """
    Rescale minimum shares of production to meet a minimum total.
    
    Assumes targets in dict_fuel_info have total of 0.
    
    
    Function Arguments
    ------------------
    - df_energy_trajectories: data frame containing INEN fractions
        to be modified
    - dict_fuel_info: dictionary mapping fuel to two keys:
        * "target": target fraction to apply to all time periods
        * "cats": catgories to apply to
    - regions: regions to apply to
    """
    # some initialization
    df_energy_trajectories = df_energy_trajectories_in.copy()
    attr_enfu = sa.model_attributes.dict_attributes.get("cat_fuel")
    attr_ind = sa.model_attributes.dict_attributes.get("cat_industry")
    cats = attr_ind.key_values if (cats_to_apply is None) else cats_to_apply
    
    # get target total
    targ_total = [v for k, v in dict_fuel_info.items() if k in attr_enfu.key_values]
    targ_total = float(sf.vec_bounds(sum(targ_total), (0, 1)))
    scalar_existing = 1 - targ_total
    
    dict_inen_fracs = model_energy.dict_inen_fuel_categories_to_fuel_variables
    regions = (
        sorted(list(df_energy_trajectories[field_iso].unique()))
        if regions is None
        else regions
    )
    df_grouped_by_region = df_energy_trajectories.groupby([field_iso])
    
    df_out = []
    
    for region, df in df_grouped_by_region:
        
        if region in regions:
            for fuel in dict_inen_fracs.keys():
                
                dict_vars_cur = dict_inen_fracs.get(fuel)    
                dict_fuel_info_cur = dict_fuel_info.get(fuel)
                
                modvar = dict_vars_cur.get("fuel_fraction")
                
                # get categories, check against spcificationin Model Attributes, then convert to fields
                cats_fields = sa.model_attributes.get_variable_categories(modvar)
                cats_fields = (
                    [x for x in cats if x in cats_fields]
                    if cats_fields is not None
                    else None
                )
                fields = sa.model_attributes.build_varlist(
                    None,
                    modvar,
                    restrict_to_category_values = cats_fields
                )
                
                # only adjust fields that are present
                fields_adj = [x for x in fields if x in df.columns]
                df[fields_adj] = (
                    np.array(df[fields_adj])*scalar_existing
                    if fuel not in dict_fuel_info.keys()
                    else dict_fuel_info.get(fuel)
                )
            
        df_out.append(df)
        
    df_out = (
        pd.concat(df_out, axis = 0)
        .sort_values(by = [field_iso, field_time_period])
        .reset_index(drop = True)
    )
        
      
    return df_out



###
###    TEMP: ADD IN SCOE ELASTS -- MOVE TO BUILD TRANSFORMATIONS FILE FUNCTION/PROCESS
###


def get_energy_data(
) -> pd.DataFrame:
    fps_read = [
        # Elasticity of SCOE energy consumption to GDP/Capita
        sa.fp_csv_scoe_elasticity_of_energy_consumption,
        # SCOE Energy Consumption Scalar
        sa.fp_csv_scoe_consumption_scalar,
        # Initial SCOE Energy consumption
        sa.fp_csv_scoe_initial_energy_consumption,
    ]
    
    df_out = None
    
    
    for fp in fps_read:
        
        df_tmp = pd.read_csv(fp)
        
        df_out = (
            df_tmp
            if df_out is None
            else pd.merge(df_out, df_tmp, how = "inner")
        )

    df_out = time_periods.years_to_tps(
        df_out,
        field_year = field_year
    )

    
    return df_out


def get_ippu_data(
) -> pd.DataFrame:
    fps_read = [
        # Elasticity of industrial production to GDP
        sa.fp_csv_elasticity_of_industrial_production,
        # Flourinated Compound emission factors
        sa.fp_csv_ippu_fc_efs,
        # Fraction of cement from clinker
        sa.fp_csv_ippu_frac_cement_clinker,
        # Industrial Production Scalar
        sa.fp_csv_industrial_production_scalar,
        # Initial Industrial production
        sa.fp_csv_initial_industrial_production,
        # Net Imports clinker
        sa.fp_csv_ippu_net_imports_cement_clinker,
        
    ]
    
    df_out = None
    
    
    for fp in fps_read:
        
        df_tmp = pd.read_csv(fp)
        
        df_out = (
            df_tmp
            if df_out is None
            else pd.merge(df_out, df_tmp, how = "inner")
        )

    df_out = time_periods.years_to_tps(
        df_out,
        field_year = field_year
    )

    
    return df_out


df_ener_overwrites = get_energy_data()
df_ippu_overwrites = get_ippu_data()







######################
###                ###
###    END TEMP    ###
###                ###
######################

if True:
    # load in tables and make some quick modifications
    for k in dict_calibration_file_paths.keys():
        
        # read and clean columns
        df_read = pd.read_csv(dict_calibration_file_paths.get(k))
        dict_rnm = dict((x, x.lower()) for x in df_read.columns if x != x.lower())
        df_read.rename(columns = dict_rnm, inplace = True)
        
        # drop ISOs that are undefiuned
        df_read = df_read[df_read[field_iso].isin(all_iso)].reset_index(drop = True)
        # add time period if not present
        if attr_time_period.key not in df_read.columns:
            df_read[attr_time_period.key] = df_read[field_year].apply(time_periods.year_to_tp)
        
        
        ###
        ###  TEMPORARY: DROP OLD FUELS
        ###
        if True:
            fields_drop = [x for x in df_read.columns if ("gas_furnace" in x) or ("gas_petroleum_liquid" in x)]
            df_read.drop(fields_drop, axis = 1, inplace = True) if (len(fields_drop) > 0) else None
        
        
        ###
        ###  TEMPORARY: OVERWRITE AFOLU CLIMATE DATA
        ###
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_climate,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
        
        
        ###
        ###    TEMPORARY: OVERWRITE OTHER KEY ENERGY DATA
        ###
        if True and not running_out_of_lac:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_overwrite,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
  
            
        
           
        ###
        ###    TEMPORARY: OVERWRITE FUEL COSTS
        ###
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_fuel_costs,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
            
            
        
        ###
        ###    TEMPORARY: OVERWRITE MAX CAPACITY
        ###
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_max_cap,
                fields_index = [sa.model_attributes.dim_time_period],
                overwrite_only = False
            )
            
            
        
            
            
        ###
        ###    TEMPORARY: RENAME FROM CALIBRATED FILE
        ###
        
        if False:
            dict_repl = {
                "gas_petroleum_liquid": "hydrocarbon_gas_liquids",
                "gas_furnace": "furnace_gas"
            }

            dict_rnm = {}

            for substr in dict_repl.keys():
                dict_rnm_cur = dict(
                    (x, x.replace(substr, dict_repl.get(substr))) 
                    for x in df_read.columns if substr in x
                )

                dict_rnm.update(dict_rnm_cur)

            df_read.rename(columns = dict_rnm, inplace = True)

        
        ####
        ####   TEMPORARY OVERWRITE OF RESIDUAL CAPACITY
        ####

        print("TEMPORARY SOLUTION: REPLACE THIS CODE ONCE RESIDUAL CAPACITIES ARE FIXED")
        
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_residual_capacity_overwrite,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
            
        
        ####
        ####   TEMPORARY OVERWRITE OF ELECTRICAL TRANSMISSION LOSSES
        ####

        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_transmission_losses,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
        

        ####
        ####   TEMPORARY OVERWRITE OF INITIAL PRODUCTION
        ####
        if False:
            # DEPRECATED AS OF 26042023--INTEGRATED INTO RBD PIPELINE
            df_read = sf.match_df_to_target_df(
                df_read,
                df_prodvalues,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
        

        ###
        ###    TEMPORARY: OVERWRITE EFFICIENCY FACTORS FOR RENEWABLES IN ENTC
        ###
        
        if True:
            fields = sa.model_attributes.build_varlist(
                "Energy Technology",
                "Technology Efficiency of Fuel Use",
                restrict_to_category_values = ["pp_hydropower", "pp_geothermal", "pp_solar", "pp_wind", "pp_ocean"]
            )
            df_read[fields] = 1
        

        ###
        ###    TEMPORARY: OVERWRITE EFFICIENCY FACTORS FOR ELECTRICITY HEAT IN SCOE/INEN
        ###
        
        if True:
            df_effic_factors_merge = sf.explode_merge(
                df_effic_factors,
                df_read[[field_iso]].drop_duplicates()
            )
            df_read = sf.match_df_to_target_df(
                df_read,
                df_effic_factors_merge,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
        

        ###
        ###    TEMPORARY: ADD HYDROPOWER GENERATION CONSTRIANT
        ###

        if False:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_hydro_cap_synthetic,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
        

        ###
        ###    TEMPORARY: OVERWRITE MinShareProduction for Baseline
        ###

        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_min_share_production,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
            #print(df_read[df_read[field_iso].isin(["BRA"])][[x for x in df_read.columns if "min_share_prod" in x]].head())


        # filter time periods and do some field cleaning
        if field_time_period not in df_read.columns:
            df_read[field_time_period] = np.array(np.array(df_read[field_year]) - 2015).astype(int)
            df_read = df_read[
                df_read[field_time_period].isin(attr_time_period.key_values)
            ].reset_index(drop = True)
        df_read = df_read[df_read[field_time_period] >= 0].reset_index(drop = True)
        df_read[field_region] = [dict_replace_iso.get(x.lower()) for x in list(df_read[field_iso])]

        # fields missing from input file: take from fake data
        fields_missing = list(set(df_fake_data.columns) - set(df_read.columns))
        fields_eliminate = list((set(df_read.columns) - set(df_fake_data.columns)) - set(fields_drop) - set(["strategy_id"]))

        df_read = pd.merge(
            df_read, 
            df_fake_data[[field_time_period] + fields_missing],
            on = [field_time_period],
            how = "left"
        )
        
        
        ###
        ###    TEMPORARY: OVERWRITE VALUES WITH FIXED VALUES SET IN DICTIONARY
        ###
        
        if True:
            dict_fields_to_value = dict(
                (x, 1.0) 
                for x in df_read.columns 
                if x.startswith("demscalar")
                and x not in (list(df_ippu_overwrites.columns) + list(df_ener_overwrites.columns))
            )
            
            for fld in dict_fields_to_value.keys():
                val = dict_fields_to_value.get(fld)
                df_read[fld] = val
                
            print("NOTE: OVERWRITE DEMSCALARS AS 1; SHOULD BE INSTITUTED IN DATA FILE")
                
                
        
        ###
        ###  TEMPORARY OVERWRITE OF WOOD FUEL FRACTIONS FROM RECYCLED WOOD 
        ###

        if False:
            for field_old in dict_cp_wood.keys():
                field_new = dict_cp_wood.get(field_old)
                df_read[field_new] = list(df_read[field_old])

            
            
            
        ###
        ###  TEMPORARY: OVERWRITE KEY IPPU AND CLINKER DATA
        ###
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_ippu_overwrites,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
            
            
        
        ###
        ###  TEMPORARY: OVERWRITE KEY ENERGY CONSUMPTION DATA
        ###
        if True:
            df_read = sf.match_df_to_target_df(
                df_read,
                df_ener_overwrites,
                fields_index = [sa.model_attributes.dim_time_period, field_iso],
                overwrite_only = False
            )
             



        # filter out regions with invalid time period specifications
        df_read = check_time_periods(
            df_read,
            attr_time_period,
            [field_iso]
        )   
        # update available iso codes
        dict_isos.update({k: set(df_read[field_iso])})
        
        # fix region issue
        df_read = (
            pd.merge(
                df_read.drop([field_region], axis = 1),
                attr_region.table[[attr_region.key, "iso_alpha_3"]].rename(
                    columns = {
                        "iso_alpha_3": field_iso,
                        attr_region.key: field_region
                    }
                )
            )
            
        )

        dict_calibration_tables.update({k: df_read})
        set_merge = set(df_read[field_region])
        dict_sets.update({k: set_merge})
        all_regions = set_merge if (all_regions is None) else (all_regions & set_merge)
        
        
        # write file for templatization input
        fp_write = os.path.join(dir_template_inputs, f"template_input_{k}.csv")
        dict_template_file_paths.update({k: fp_write})
        
        print(f"Writing sector '{k}' to {fp_write}...")
        dict_calibration_tables.get(k).to_csv(
            fp_write,
            index = None,
            encoding = "UTF-8"
        )
        print("Done.\n\n")

    attr_region = sa.model_attributes.dict_attributes.get("region")
    all_regions = sorted(list(set(attr_region.key_values) & all_regions))





TEMPORARY SOLUTION: REPLACE THIS CODE ONCE RESIDUAL CAPACITIES ARE FIXED
NOTE: OVERWRITE DEMSCALARS AS 1; SHOULD BE INSTITUTED IN DATA FILE
Writing sector 'af' to /Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo/template_inputs/template_input_af.csv...
Done.


TEMPORARY SOLUTION: REPLACE THIS CODE ONCE RESIDUAL CAPACITIES ARE FIXED
NOTE: OVERWRITE DEMSCALARS AS 1; SHOULD BE INSTITUTED IN DATA FILE
Writing sector 'ce' to /Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo/template_inputs/template_input_ce.csv...
Done.


TEMPORARY SOLUTION: REPLACE THIS CODE ONCE RESIDUAL CAPACITIES ARE FIXED
NOTE: OVERWRITE DEMSCALARS AS 1; SHOULD BE INSTITUTED IN DATA FILE
Writing sector 'en' to /Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo/template_inputs/template_input_en.csv...
Done.


TEMPORARY SOLUTION: REPLACE THIS CODE ONCE RESIDUAL CAPACITIES ARE FIXED
NOTE: OVERWRITE

### Use TransformationsEnergy to build transformations

In [45]:
importlib.reload(ma)
importlib.reload(sa)
importlib.reload(tbc)
importlib.reload(tbe)
importlib.reload(dtc)
importlib.reload(dte)


year_0_ramp = 2025
df_input = dict_calibration_tables.get("en").copy()


# set some parameters

dict_config_te = {
    "categories_entc_max_investment_ramp": [
        "pp_hydropower",
        "pp_nuclear"
    ],
    "categories_entc_renewable": [
        "pp_geothermal",
        "pp_hydropower",
        "pp_ocean",
        "pp_solar",
        "pp_wind"
    ],
    "categories_inen_high_heat": [
        "cement", 
        "chemicals", 
        "glass", 
        "lime_and_carbonite", 
        "metals"
    ],
    "dict_entc_renewable_target_msp": {
        "pp_solar": 0.15,
        "pp_geothermal": 0.1,
        "pp_wind": 0.15
    },
    "frac_inen_high_temp_elec_hydg": 0.5*0.45,
    "frac_inen_low_temp_elec": 0.95*0.45,
    "n_tp_ramp": None,
    "vir_renewable_cap_delta_frac": 0.01,
    "vir_renewable_cap_max_frac": 0.05,
    "year_0_ramp": year_0_ramp
}



transformations_circular_economy = dtc.TransformationsCircularEconomy(
    sa.model_attributes,
    dict_config_te,
    df_input = df_input,
    field_region = "nation",
    logger = logger
)


transformations_energy = dte.TransformationsEnergy(
    sa.model_attributes,
    dict_config_te,
    sa.dir_jl,
    sa.dir_ref_nemo,
    df_input = df_input,
    field_region = "nation",
    logger = logger
)



    
# initialze output components
#df_out = transformations_energy.build_strategies_long()#(strategies = [0, 3003])
    


In [46]:
df_out = transformations_circular_economy.build_strategies_long()#(strategies = [0, 3003])

2023-05-25 20:29:36,700 - INFO - TransformationsCircularEconomy.build_strategies_long() starting build of 16 strategies...
2023-05-25 20:29:37,581 - INFO - 	Successfully built transformation strategy_id = 2001 ('TRWW: ALL') in 0.88 seconds.


YEAH - 1354356


2023-05-25 20:29:37,980 - INFO - 	Successfully built transformation strategy_id = 2002 ('TRWW: Increase biogas capture') in 0.4 seconds.


YEAH - 1354356


2023-05-25 20:29:38,333 - INFO - 	Successfully built transformation strategy_id = 2003 ('TRWW: Increase septic compliance') in 0.35 seconds.


...

2023-05-25 20:29:47,219 - INFO - 	Successfully built transformation strategy_id = 2015 ('WASO: Waste reduction') in 0.76 seconds.


YEAH - 1354356


2023-05-25 20:29:51,950 - INFO - 	Successfully built transformation strategy_id = 2016 ('CE: All Circular Economy transformations') in 4.73 seconds.


YEAH - 1354356


2023-05-25 20:29:52,907 - INFO - TransformationsCircularEconomy.build_strategies_long() build complete in 16.21 seconds.


In [44]:

df_out = tbc.transformation_waso_increase_landfilling(
    df_input,
    1.0,
    transformations_circular_economy.vec_implementation_ramp,
    transformations_circular_economy.model_attributes,
    field_region = transformations_circular_economy.key_region,
    model_circecon = transformations_circular_economy.model_circecon,
    strategy_id = 0
)


sa.model_attributes.get_standard_variables(
    df_out,
    model_circecon.modvar_waso_frac_nonrecycled_landfill
).iloc[0:38]



,frac_waso_non_recycled_landfilled
0,0.600
1,0.600
2,0.600
3,0.600
4,0.600
5,0.600
6,0.600
7,0.600
8,0.600
9,0.600


In [ ]:
### importlib.reload(mc)
model_circecon = mc.CircularEconomy(sa.model_attributes)
#import transformations_base_general as tbg
importlib.reload(tbg)
importlib.reload(tbc)

df_out = tbc.transformation_trww_increase_gas_capture(
    df_input,
    {
        "treated_advanced_anaerobic": 0.85,
        "treated_secondary_anaerobic": 0.5,
    },
    transformations_circular_economy.vec_implementation_ramp,
    sa.model_attributes,
)

sa.model_attributes.get_standard_variables(
    df_out,
    model_circecon.modvar_trww_rf_biogas_recovered
).iloc[0:36]

#df_out[[x for x in df_out.columns if x.startswith("frac_wali_ww_industrial_treatment_path_")]].iloc[0:36]#.sum(axis = 1)

In [ ]:
sa.model_attributes.assign_keys_from_attribute_fields(
    self.subsec_name_wali,
    "cat_"
)

In [76]:
importlib.reload(ma)
importlib.reload(sa)
importlib.reload(dtc)

transformations_circular_economy = dtc.TransformationsCircularEconomy(
    sa.model_attributes,
    dict_config_te,
    df_input = df_input,
    field_region = "nation",
    logger = logger
)

df_tst = transformations_circular_economy.build_strategies_long()

2023-05-25 13:57:37,725 - INFO - TransformationsCircularEconomy.build_strategies_long() starting build of 8 strategies...
2023-05-25 13:57:37,725 - INFO - TransformationsCircularEconomy.build_strategies_long() starting build of 8 strategies...
2023-05-25 13:57:38,763 - INFO - 	Successfully built transformation strategy_id = 2010 ('WASO: Waste reduction') in 1.04 seconds.
2023-05-25 13:57:38,763 - INFO - 	Successfully built transformation strategy_id = 2010 ('WASO: Waste reduction') in 1.04 seconds.


YEAH - 1354356


2023-05-25 13:57:39,050 - INFO - 	Successfully built transformation strategy_id = 2011 ('WASO: Increase recycling') in 0.29 seconds.
2023-05-25 13:57:39,050 - INFO - 	Successfully built transformation strategy_id = 2011 ('WASO: Increase recycling') in 0.29 seconds.


YEAH - 1354356


2023-05-25 13:57:39,344 - INFO - 	Successfully built transformation strategy_id = 2012 ('WASO: Increase composting and biogas') in 0.29 seconds.
2023-05-25 13:57:39,344 - INFO - 	Successfully built transformation strategy_id = 2012 ('WASO: Increase composting and biogas') in 0.29 seconds.


...

2023-05-25 13:57:42,517 - INFO - 	Successfully built transformation strategy_id = 2016 ('WASO: Incineration for energy production') in 0.25 seconds.
2023-05-25 13:57:42,517 - INFO - 	Successfully built transformation strategy_id = 2016 ('WASO: Incineration for energy production') in 0.25 seconds.


YEAH - 1354356


2023-05-25 13:57:42,873 - INFO - 	Successfully built transformation strategy_id = 2017 ('WASO: Biogas for energy production') in 0.35 seconds.
2023-05-25 13:57:42,873 - INFO - 	Successfully built transformation strategy_id = 2017 ('WASO: Biogas for energy production') in 0.35 seconds.


YEAH - 1354356


2023-05-25 13:57:43,301 - INFO - TransformationsCircularEconomy.build_strategies_long() build complete in 5.58 seconds.
2023-05-25 13:57:43,301 - INFO - TransformationsCircularEconomy.build_strategies_long() build complete in 5.58 seconds.


In [70]:
model_circecon = mc.CircularEconomy(sa.model_attributes)
df_select = df_tst[
    df_tst["strategy_id"].isin([2013])
    & df_tst["iso_code3"].isin(["BRA"])
]

if True:
    df_print = sa.model_attributes.get_standard_variables(
        df_select,
        #model_circecon.modvar_waso_frac_landfill_gas_ch4_to_energy
        model_circecon.modvar_waso_frac_recovered_for_energy_incineration_isw
        #model_circecon.modvar_waso_frac_recycled
    )
else:
    df_print = df_select[
        [
            "frac_waso_non_recycled_landfilled", 
            "frac_waso_non_recycled_incinerated", 
            "frac_waso_non_recycled_open_dump"
        ]
    ]

df_print.tail()

,frac_waso_isw_incinerated_recovered_for_energy
31,0.05
32,0.05
33,0.05
34,0.05
35,0.05


In [196]:
if True:
    fp_out = dict_template_file_paths.get("en").replace(".csv", "_with_transformations.csv")
    print(f"writing to output at {fp_out}...")
    df_out.to_csv(
        fp_out,
        index = None,
        encoding = "UTF-8"
    )

print("Done.")


writing to output at /Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo/template_inputs/template_input_en_with_transformations.csv...
Done.


In [203]:
frac_shift_hh_elec = transformations_energy.frac_inen_low_temp_elec + transformations_energy.frac_inen_high_temp_elec_hydg
frac_shift_hh_elec /= transformations_energy.frac_inen_shift_denom

frac_shift_hh_hydrogen = transformations_energy.frac_inen_high_temp_elec_hydg
frac_shift_hh_hydrogen /= transformations_energy.frac_inen_shift_denom
frac_shift_hh_hydrogen, frac_shift_hh_elec




['cement', 'chemicals', 'glass', 'lime_and_carbonite', 'metals']

In [242]:
importlib.reload(adt)
df_test = df_input[df_input["iso_code3"].isin(["BRA"])].reset_index(drop = True)
df_test_out_hi  = tbe.transformation_inen_shift_modvars(
    df_test,
    transformations_energy.frac_inen_shift_denom,
    transformations_energy.vec_implementation_ramp, 
    transformations_energy.model_attributes,
    categories = transformations_energy.cats_inen_high_heat,
    dict_modvar_specs = {
        transformations_energy.model_energy.modvar_inen_frac_en_electricity: frac_shift_hh_elec,
        transformations_energy.model_energy.modvar_inen_frac_en_hydrogen: frac_shift_hh_hydrogen,
    },
    field_region = transformations_energy.key_region,
    model_energy = transformations_energy.model_energy,
    strategy_id = 2
)


df_test_out_lo = tbe.transformation_inen_shift_modvars(
    df_test,
    transformations_energy.frac_inen_shift_denom,
    transformations_energy.vec_implementation_ramp, 
    transformations_energy.model_attributes,
    categories = transformations_energy.cats_inen_not_high_heat,
    dict_modvar_specs = {
        transformations_energy.model_energy.modvar_inen_frac_en_electricity: 1.0
    },
    field_region = transformations_energy.key_region,
    #magnitude_relative_to_baseline = True,
    model_energy = transformations_energy.model_energy,
    strategy_id = 2
)

In [243]:
#attr_strat.table[attr_strat.table["strategy_id"].isin(range(3001, 3005))]
df_test_out_hi[[x for x in df_test_out.columns if x.startswith("frac_inen_energy_cement")]].iloc[[0, 1, 2, 33, 34, 35]]

,frac_inen_energy_cement_coal,frac_inen_energy_cement_coke,frac_inen_energy_cement_diesel,frac_inen_energy_cement_electricity,frac_inen_energy_cement_furnace_gas,frac_inen_energy_cement_gasoline,frac_inen_energy_cement_hydrocarbon_gas_liquids,frac_inen_energy_cement_hydrogen,frac_inen_energy_cement_kerosene,frac_inen_energy_cement_natural_gas,frac_inen_energy_cement_oil,frac_inen_energy_cement_solar,frac_inen_energy_cement_solid_biomass
0,0.013186,0.001465,0.060630,0.105042,0.0,0.060630,0.0,0.000,0.0,0.147624,0.282938,0.052614,0.275871
1,0.010716,0.001191,0.058555,0.106807,0.0,0.058555,0.0,0.000,0.0,0.163260,0.273254,0.051556,0.276108
2,0.011824,0.001314,0.057253,0.105523,0.0,0.057253,0.0,0.000,0.0,0.167453,0.267180,0.050266,0.281935
33,0.002626,0.000292,0.013011,0.608699,0.0,0.013011,0.0,0.207,0.0,0.029705,0.060716,0.011170,0.053772
34,0.002185,0.000243,0.010829,0.630599,0.0,0.010829,0.0,0.216,0.0,0.024725,0.050536,0.009297,0.044757
35,0.001745,0.000194,0.008648,0.652500,0.0,0.008648,0.0,0.225,0.0,0.019744,0.040356,0.007424,0.035741


In [245]:
#attr_strat.table[attr_strat.table["strategy_id"].isin(range(3001, 3005))]
df_test_out_lo[[x for x in df_test_out.columns if x.startswith("frac_inen_energy_cement")]].iloc[[0, 1, 2, 33, 34, 35]]

,frac_inen_energy_cement_coal,frac_inen_energy_cement_coke,frac_inen_energy_cement_diesel,frac_inen_energy_cement_electricity,frac_inen_energy_cement_furnace_gas,frac_inen_energy_cement_gasoline,frac_inen_energy_cement_hydrocarbon_gas_liquids,frac_inen_energy_cement_hydrogen,frac_inen_energy_cement_kerosene,frac_inen_energy_cement_natural_gas,frac_inen_energy_cement_oil,frac_inen_energy_cement_solar,frac_inen_energy_cement_solid_biomass
0,0.013186,0.001465,0.060630,0.105042,0.0,0.060630,0.0,0.0,0.0,0.147624,0.282938,0.052614,0.275871
1,0.010716,0.001191,0.058555,0.106807,0.0,0.058555,0.0,0.0,0.0,0.163260,0.273254,0.051556,0.276108
2,0.011824,0.001314,0.057253,0.105523,0.0,0.057253,0.0,0.0,0.0,0.167453,0.267180,0.050266,0.281935
33,0.012751,0.001417,0.063183,0.104982,0.0,0.063183,0.0,0.0,0.0,0.144258,0.294852,0.054243,0.261133
34,0.012751,0.001417,0.063183,0.104982,0.0,0.063183,0.0,0.0,0.0,0.144258,0.294852,0.054243,0.261133
35,0.012751,0.001417,0.063183,0.104982,0.0,0.063183,0.0,0.0,0.0,0.144258,0.294852,0.054243,0.261133


In [244]:
df_test_out_transf = transformations_energy.inen_all(df_test)
df_test_out_transf[[x for x in df_test_out_transf.columns if x.startswith("frac_inen_energy_cement")]].iloc[[0, 1, 2, 33, 34, 35]]

,frac_inen_energy_cement_coal,frac_inen_energy_cement_coke,frac_inen_energy_cement_diesel,frac_inen_energy_cement_electricity,frac_inen_energy_cement_furnace_gas,frac_inen_energy_cement_gasoline,frac_inen_energy_cement_hydrocarbon_gas_liquids,frac_inen_energy_cement_hydrogen,frac_inen_energy_cement_kerosene,frac_inen_energy_cement_natural_gas,frac_inen_energy_cement_oil,frac_inen_energy_cement_solar,frac_inen_energy_cement_solid_biomass
0,0.013186,0.001465,0.060630,0.105042,0.0,0.060630,0.0,0.000,0.0,0.147624,0.282938,0.052614,0.275871
1,0.010716,0.001191,0.058555,0.106807,0.0,0.058555,0.0,0.000,0.0,0.163260,0.273254,0.051556,0.276108
2,0.011824,0.001314,0.057253,0.105523,0.0,0.057253,0.0,0.000,0.0,0.167453,0.267180,0.050266,0.281935
33,0.002626,0.000292,0.013011,0.608699,0.0,0.013011,0.0,0.207,0.0,0.029705,0.060716,0.011170,0.053772
34,0.002185,0.000243,0.010829,0.630599,0.0,0.010829,0.0,0.216,0.0,0.024725,0.050536,0.009297,0.044757
35,0.001745,0.000194,0.008648,0.652500,0.0,0.008648,0.0,0.225,0.0,0.019744,0.040356,0.007424,0.035741


In [221]:
trans_try = sc.Transformation(
    "INEN: All Industrial Energy transformations", 
    [
        transformations_energy.transformation_inen_fuel_switch_low_and_high_temp, # use instead of both functions to avoid incorrect results w/func composition
        #self.transformation_inen_maximize_efficiency_energy,
        #self.transformation_inen_maximize_efficiency_production
    ], 
    attr_strat
)

df_test_out_transf = trans_try(df_test)
df_test_out_transf[[x for x in df_test_out_transf.columns if x.startswith("frac_inen_energy_cement")]].iloc[[0, 1, 2, 33, 34, 35]]



df_test_out = transformations_energy.transformation_inen_fuel_switch_low_and_high_temp(
    df_test,
    strat = 2
)
df_test_out[[x for x in df_test_out.columns if x.startswith("frac_inen_energy_cement")]].iloc[[0, 1, 2, 33, 34, 35]]

,frac_inen_energy_cement_coal,frac_inen_energy_cement_coke,frac_inen_energy_cement_diesel,frac_inen_energy_cement_electricity,frac_inen_energy_cement_furnace_gas,frac_inen_energy_cement_gasoline,frac_inen_energy_cement_hydrocarbon_gas_liquids,frac_inen_energy_cement_hydrogen,frac_inen_energy_cement_kerosene,frac_inen_energy_cement_natural_gas,frac_inen_energy_cement_oil,frac_inen_energy_cement_solar,frac_inen_energy_cement_solid_biomass
0,0.013186,0.001465,0.060630,0.105042,0.0,0.060630,0.0,0.000000,0.0,0.147624,0.282938,0.052614,0.275871
1,0.010716,0.001191,0.058555,0.106807,0.0,0.058555,0.0,0.000000,0.0,0.163260,0.273254,0.051556,0.276108
2,0.011824,0.001314,0.057253,0.105523,0.0,0.057253,0.0,0.000000,0.0,0.167453,0.267180,0.050266,0.281935
33,0.000291,0.000032,0.001441,0.952635,0.0,0.001441,0.0,0.026959,0.0,0.003289,0.006722,0.001237,0.005954
34,0.000172,0.000019,0.000850,0.968465,0.0,0.000850,0.0,0.019491,0.0,0.001941,0.003968,0.000730,0.003514
35,0.000088,0.000010,0.000434,0.982542,0.0,0.000434,0.0,0.011304,0.0,0.000992,0.002027,0.000373,0.001796


In [229]:
attr_inds = sa.model_attributes.get_attribute_table(sa.model_attributes.subsec_name_ippu)
sub = set(transformations_energy.cats_inen_not_high_heat)
len(sub & set(attr_inds.key_values)) - len(sub)

0

In [365]:
importlib.reload(mc)
model_circecon = mc.CircularEconomy(sa.model_attributes)
importlib.reload(adt)

"""
sa.model_attributes.get_standard_variables(
    t,
    model_circecon.modvar_waso_waste_per_capita_scalar,
)
"""


t2 = 
sa.model_attributes.get_standard_variables(
    t2,
    model_circecon.modvar_waso_frac_nonrecycled_incineration,
)

t2[["frac_waso_non_recycled_landfilled", "frac_waso_non_recycled_open_dump", "frac_waso_non_recycled_incinerated"]]

here!:	True and True


,frac_waso_non_recycled_landfilled,frac_waso_non_recycled_open_dump,frac_waso_non_recycled_incinerated
0,0.600,0.300,0.100
1,0.600,0.300,0.100
2,0.600,0.300,0.100
3,0.600,0.300,0.100
4,0.600,0.300,0.100
5,0.600,0.300,0.100
6,0.600,0.300,0.100
7,0.600,0.300,0.100
8,0.600,0.300,0.100
9,0.600,0.300,0.100


In [348]:
model_circecon.modvars_waso_frac_non_recyled_pathways

['Fraction of Non-Recycled Solid Waste Incinerated',
 'Fraction of Non-Recycled Solid Waste Landfilled',
 'Fraction of Non-Recycled Solid Waste Open Dumps']

In [74]:
?tbe.transformation_general_shift_fractions_from_modvars

Signature:
tbe.transformation_general_shift_fractions_from_modvars(
    df_input: pandas.core.frame.DataFrame,
    magnitude: float,
    modvars: List[str],
    dict_modvar_specs: Dict[str, float],
    vec_ramp: numpy.ndarray,
    model_attributes: model_attributes.ModelAttributes,
    categories: Optional[List[str]] = None,
    field_region: str = 'nation',
    magnitude_relative_to_baseline: bool = False,
    regions_apply: Optional[List[str]] = None,
    strategy_id: Optional[int] = None,
) -> pandas.core.frame.DataFrame
Docstring:
Implement fractional swap transformations

Function Arguments
------------------
- df_input: input data frame containing baseline trajectories
- magnitude: target magnitude of mixture (summed across target categories)
- modvars: list of model variables used to constitute fractions
- dict_modvar_specs: dictionary of targets modvars to shift into (assumes
    that will take from others). Maps from modvar to fraction of magnitude.
    Sum of values must == 1

In [ ]:
df_test = df_input[df_input["iso_code3"].isin(["BRA"])]
tbe.transformation_general_shift_fractions_from_modvars(
    df_test, 
    0.5,
    
)

In [251]:
dict_a = {"this": 0.5}

x = dict_a.get("this")

x *= 4

dict_a

{'this': 0.5}

In [262]:
arr = np.ones((6, 3))
arr[:,2] = 1.5

arr[5, 1] = 1.2
arr


sf.vec_bounds(arr, (0, 1.4))

array([[1. , 1. , 1.4],
       [1. , 1. , 1.4],
       [1. , 1. , 1.4],
       [1. , 1. , 1.4],
       [1. , 1. , 1.4],
       [1. , 1.2, 1.4]])

In [254]:
sf.vec_bounds(
    
)

Signature: sf.vec_bounds(vec, bounds: tuple, cycle_vector_bounds_q: bool = False)
Docstring:
Bound a vector vec within a range set within 'bounds'.

Function Arguments
------------------
- vec: list or np.ndarray of values to bound
- bounds: tuple (single bound) or list vec specifying element-wise bounds. 
    NOTE: only works if

    vec.shape = (len(vec), ) == (len(bounds), )

Keyword Arguments
-----------------
- cycle_vector_bounds_q: cycle bounds if there is a mismatch and the bounds 
    are entered as a vector
File:      ~/Documents/Projects/git_jbus/lac_decarbonization/python/support_functions.py
Type:      function
